In [1]:
'''
Coding: PYTHON UTF-8
Created On: 2023-04-28 12:39:30
Author: Putu Hendra Widyadharma
=== filter .pha using wadati result from makeWADATI 
'''

'\nCoding: PYTHON UTF-8\nCreated On: 2023-04-28 12:39:30\nAuthor: Putu Hendra Widyadharma\n=== filter .pha using wadati result from makeWADATI \n'

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from localfunction import *
pd.options.mode.chained_assignment = None

In [3]:
path = 'G:\\My Drive\\IUGG Seminar\\data\\data_poster\\'
fname = 'arrivals.dat'
fname2 = 'temppha_arrivals.txt' #temppha
fname3 = 'tempwadati_arrivals.txt' #tempwadati
deviation = 3

In [4]:
df = readabsolute(path+fname)
dfphase = pd.read_csv(path+fname2,delim_whitespace=True)
dfwadati = pd.read_csv(path+fname3,delim_whitespace=True)

In [5]:
dfwadati

,sta,tp,ts,ts-tp,dist_from_event,event_id
0,BM94,4.84,7.04,2.20,9.933215,1
1,BM92,4.00,4.96,0.96,11.640506,1
2,BM95,3.71,5.46,1.75,14.469164,1
3,BM96,4.56,6.93,2.37,15.896357,1
4,SCJM,5.18,8.51,3.33,16.594396,1
...,...,...,...,...,...,...
3250,BM92,1.67,2.40,0.73,3.202982,484
3251,BM94,1.51,2.53,1.02,4.660364,484
3252,CCJM,3.79,6.98,3.19,21.085636,484
3253,CBJM,7.49,11.09,3.60,26.122501,484


In [14]:
# hitung dan buat persamaan pembatas 
slope, intercept = np.polyfit(dfwadati['tp'].astype(float),dfwadati['ts-tp'].astype(float),1)

# manual slope
# slope = 0.589
# intercept = 0.9170

# predict function
predict = np.poly1d([slope, intercept])
# predict with deviation
predictup = np.poly1d([slope,intercept+deviation])
predictdown = np.poly1d([slope,intercept-deviation])

# hitung R^2 menguji kecocokan regresi
r2 = r2_score(dfwadati['ts-tp'],predict(dfwadati['tp']))
print('VP/VS = ' + str(slope + 1))
print(f"R-square = {r2}")

VP/VS = 1.589
R-square = 0.4652056560376372


In [15]:
### dfwadati processing
# fungsi seleksi data berdasarkan batas atas dan bawah deviasi
def selection_data(row):
    if (row['ts-tp'] <= predictup(row['tp'])) and (row['ts-tp'] >= predictdown(row['tp'])):
        return True
    else:
        return False
# apply perhitungan
dfwadati['result'] = dfwadati.apply(selection_data,axis=1)
# select wadati hanya pada event yang perlu diubah
dfwadati = dfwadati[dfwadati['event_id'].isin(dfwadati[dfwadati['result'] == False]['event_id'].unique())]

In [16]:
# index search untuk event_id yang perlu disesuaikan menurut wadati 
editable_idx = df[df[0] == '#'][14].astype('int64').isin(dfwadati['event_id'].unique())
editable_idx = editable_idx[editable_idx==True].index
editable_idx

Int64Index([   0, 2690, 2724, 4245, 5199, 5233, 5295, 5343, 5369, 5478, 5500,
            5530, 5565, 5589, 5627, 5657, 5675, 5711, 5730, 5797, 5830, 6057,
            6113, 6159, 6178, 6311, 6673, 6692, 6779, 6874, 6923, 7023, 7071,
            7117, 7145, 7185, 7219, 7238, 7289, 7391, 7424, 7453, 7557, 7664,
            7695, 7707, 7764, 7786, 7811, 7919, 7963, 8003, 8100, 8196, 8209,
            8366, 8392, 8490, 8606, 8675, 8862, 8911, 8930, 9051, 9085, 9141,
            9214, 9237, 9279, 9332, 9522, 9537, 9604],
           dtype='int64')

In [17]:
# buat tempat data hasil
df_result = pd.DataFrame([],columns = df.columns)

# pha index iterable
idx = df[df[0] == '#'].index

for idx_ in range (len(idx)):
# for idx_ in [1099]:
# for idx_ in [0,1,2,3]:
    # buat data per kejadian gempa
    if idx_ == len(idx)-1:
        tempdf = df.iloc[idx[idx_]::]
    else:
        tempdf = df.iloc[idx[idx_]:idx[idx_+1],:]
    # cek index header terhadap editable_idx
    if tempdf.index[0] in editable_idx:
        # proses delete data fasa S stasiun yang ts-tp diluar batas deviasi
        for i, row_ in tempdf.iterrows():
            # cari fasa S dan list stasiun yang masuk dalam membuat hasil wadati buruk 
            if row_[3]=='S' and row_[0] in (dfwadati[(dfwadati['event_id'] == int(tempdf.iloc[0,14])) & (dfwadati['result'] == False)]['sta'].to_list()):
                print(f'event: {tempdf.iloc[0,14]} -- index:{row_.name} --> {row_[0]} Phase:{row_[3]} dropped')
                tempdf.drop(row_.name,inplace=True)
    else:
        pass
    # masukkan hasil ke df hasil
    df_result = pd.concat([df_result,tempdf])

event: 1 -- index:12 --> CBJI Phase:S dropped
event: 135 -- index:2692 --> BM92 Phase:S dropped
event: 137 -- index:2740 --> SSJM Phase:S dropped
event: 137 -- index:2749 --> JTJM Phase:S dropped
event: 137 -- index:2758 --> CNJI Phase:S dropped
event: 222 -- index:4267 --> CBJM Phase:S dropped
event: 272 -- index:5201 --> BM60 Phase:S dropped
event: 274 -- index:5245 --> SCJM Phase:S dropped
event: 277 -- index:5307 --> SSJM Phase:S dropped
event: 277 -- index:5315 --> CWJM Phase:S dropped
event: 279 -- index:5364 --> JTJM Phase:S dropped
event: 279 -- index:5367 --> DBJI Phase:S dropped
event: 280 -- index:5397 --> TOJI Phase:S dropped
event: 285 -- index:5493 --> CBJI Phase:S dropped
event: 286 -- index:5528 --> CIJM Phase:S dropped
event: 287 -- index:5538 --> SCJM Phase:S dropped
event: 287 -- index:5556 --> TSJM Phase:S dropped
event: 288 -- index:5587 --> TOJI Phase:S dropped
event: 289 -- index:5605 --> CBJI Phase:S dropped
event: 289 -- index:5619 --> PSJM Phase:S dropped
even

In [18]:
# perbaiki df_result dan simpan dalam bentuk pha
df_result.sort_index(inplace = True)
df_result.reset_index(inplace = True, drop = True)
df2dat(df_result,evnum = 1, path = path, fname=Path(fname).stem+'_wadatifilter.dat')
print("== data filter")
readeventphase(path+Path(fname).stem+'_wadatifilter.dat')

== data filter
! arrivals_wadatifilter.dat
---------------------------
start time        : 2022-11-23 07:10:15.290000
end time          : 2022-12-03 00:13:23.900000
minimum magnitude : 0.38
maximum magnitude : 4.39
minimum depth     : 0 km
maximum depth     : 25 km
station used      : 44
---------------------------
Event :    484
Phase :   9027
-- P  :   5875
-- S  :   3152
---------------------------
